In [15]:
import os
import cv2
import numpy as np
import torch
from natsort import natsorted
from torchvision.transforms import functional as F
from torch import nn
import math
from skimage.metrics import structural_similarity
from torchvision import transforms
from PIL import Image

In [16]:
class SRCNN(nn.Module):
    def __init__(self) -> None:
        super(SRCNN, self).__init__()
        # Feature extraction layer.
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, (9, 9), (1, 1), (4, 4)),
            nn.ReLU(True)
        )

        # Non-linear mapping layer.
        self.map = nn.Sequential(
            nn.Conv2d(64, 32, (5, 5), (1, 1), (2, 2)),
            nn.ReLU(True)
        )

        # Rebuild the layer.
        self.reconstruction = nn.Conv2d(32, 1, (5, 5), (1, 1), (2, 2))

        # Initialize model weights.
        self._initialize_weights()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self._forward_impl(x)

    # Support torch.script function.
    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        out = self.features(x)
        out = self.map(out)
        out = self.reconstruction(out)

        return out

    # The filter weight of each layer is a Gaussian distribution with zero mean and
    # standard deviation initialized by random extraction 0.001 (deviation is 0)
    def _initialize_weights(self) -> None:
        for module in self.modules():
            if isinstance(module, nn.Conv2d):
                nn.init.normal_(module.weight.data, 0.0, math.sqrt(2 / (module.out_channels * module.weight.data[0][0].numel())))
                nn.init.zeros_(module.bias.data)

        nn.init.normal_(self.reconstruction.weight.data, 0.0, 0.001)
        nn.init.zeros_(self.reconstruction.bias.data)


In [17]:
def bgr2ycbcr_torch(tensor: torch.Tensor, only_use_y_channel: bool) -> torch.Tensor:
    """
    将 BGR 彩色图像转换为 YCbCr 彩色空间，返回 torch.Tensor 类型的结果。
    Args:
        tensor: 输入图像，类型为 torch.Tensor，shape 为 [batch_size, channels, height, width]。
        only_use_y_channel: 是否只使用 Y 通道。
    Returns:
        转换后的图像，类型为 torch.Tensor，shape 与输入相同。
    """
    if only_use_y_channel:
        weight = torch.Tensor([[24.966], [128.553], [65.481]]).to(tensor)
        tensor = torch.matmul(tensor.permute(0, 2, 3, 1), weight).permute(0, 3, 1, 2) + 16.0
    else:
        weight = torch.Tensor([[24.966, 112.0, -18.214],
                               [128.553, -74.203, -93.786],
                               [65.481, -37.797, 112.0]]).to(tensor)
        bias = torch.Tensor([16, 128, 128]).view(1, 3, 1, 1).to(tensor)
        tensor = torch.matmul(tensor.permute(0, 2, 3, 1), weight).permute(0, 3, 1, 2) + bias
    tensor /= 255.
    return tensor
def ycbcr2rgb(image: np.ndarray) -> np.ndarray:
    """
    将 YCbCr 彩色图像转换为 RGB 彩色空间，返回 numpy.ndarray 类型的结果。
    Args:
        image: 输入图像，类型为 numpy.ndarray，shape 为 [height, width, channels]。
    Returns:
        转换后的图像，类型为 numpy.ndarray，shape 与输入相同。
    """
    image_dtype = image.dtype
    image *= 255.
    image = np.matmul(image, [[0.00456621, 0.00456621, 0.00456621],
                              [0, -0.00153632, 0.00791071],
                              [0.00625893, -0.00318811, 0]]) * 255.0 + [-222.921, 135.576, -276.836]

    image /= 255.
    image = image.astype(image_dtype)
    return image
def image2tensor(image: np.ndarray, range_norm: bool, half: bool) -> torch.Tensor:
    """
    将 numpy.ndarray 类型的图像转换为 torch.Tensor 类型，并进行标准化和类型转换。
    Args:
        image: 输入图像，类型为 numpy.ndarray，shape 为 [height, width, channels]。
        range_norm: 是否进行 [0, 1] 到 [-1, 1] 的标准化。
        half: 是否将类型转换为 torch.half。
    Returns:
        转换后的图像，类型为 torch.Tensor，shape 为 [batch_size, channels, height, width]。
    """
    tensor = F.to_tensor(image)
    # Scale the image data from [0, 1] to [-1, 1]
    if range_norm:
        tensor = tensor.mul(2.0).sub(1.0)
    # Convert torch.float32 image data type to torch.half image data type
    if half:
        tensor = tensor.half()
    return tensor
def bgr2ycbcr(image: np.ndarray, only_use_y_channel: bool) -> np.ndarray:
    """
    将BGR格式的图像转换为YCbCr格式的图像
    参数:
    - image: numpy数组, 形状为(H, W, 3), 存储BGR格式的图像
    - only_use_y_channel: bool型变量，True表示只使用亮度(Y)通道，False表示使用亮度(Y)、色度(Cb、Cr)通道
    返回值:
    - numpy数组，形状为(H, W, 1) 或 (H, W, 3), 存储YCbCr格式的图像，数值范围为[0, 1]    
    """
    if only_use_y_channel:
        image = np.dot(image, [24.966, 128.553, 65.481]) + 16.0
    else:
        image = np.matmul(image, [[24.966, 112.0, -18.214], [128.553, -74.203, -93.786], [65.481, -37.797, 112.0]]) + [
            16, 128, 128]

    image /= 255.
    image = image.astype(np.float32)
    return image
def ycbcr2bgr(image: np.ndarray) -> np.ndarray:
    """
    将YCbCr格式的图像转换为BGR格式的图像
    参数:
    - image: numpy数组，形状为(H, W, 3)，存储YCbCr格式的图像
    返回值:
    - numpy数组，形状为(H, W, 3)，存储BGR格式的图像，数值范围为[0, 255]
    """
    image_dtype = image.dtype
    image *= 255.
    image = np.matmul(image, [[0.00456621, 0.00456621, 0.00456621],
                              [0.00791071, -0.00153632, 0],
                              [0, -0.00318811, 0.00625893]]) * 255.0 + [-276.836, 135.576, -222.921]

    image /= 255.
    image = image.astype(image_dtype)
    return image
def tensor2image(tensor: torch.Tensor, range_norm: bool, half: bool) :
    """
    将PyTorch张量转换为numpy数组，并转换为图像格式
    参数:
    - tensor: PyTorch张量，形状为(1, 3, H, W)，存储RGB格式的图像
    - range_norm: bool型变量，表示是否需要对图像进行归一化，True表示需要进行归一化，False表示不需要进行归一化
    - half: bool型变量，表示是否需要将数据类型转换为半精度浮点数，True表示需要转换为半精度浮点数，False表示不需要转换
    返回值:
    - numpy数组，形状为(H, W, 3)，存储BGR格式的图像，数值范围为[0, 255]
    """
    if range_norm:
        tensor = tensor.add(1.0).div(2.0)
    if half:
        tensor = tensor.half()
    image = tensor.squeeze(0).permute(1, 2, 0).mul(255).clamp(0, 255).cpu().numpy().astype("uint8")
    return image
def psnr(img1, img2):
    return 10. * torch.log10(1. / torch.mean((img1 - img2) ** 2))

In [27]:
def main() -> None:
    lr_dir=f"./data/Set5"#指定包含低分辨率测试图像的目录。
    hr_dir = f"./data/Set5"#指定包含相应高分辨率测试图像的目录（仅用于评估）。
    device = torch.device("cuda", 0)#：指定计算设备为第一个可用的CUDA启用的GPU。
    upscale_factor=4#：指定图像将放大的比例因子。
    model_path='./srcnn_model/srcnn_x4-T91-7c460643.pth.tar'#指定预训练的SRCNN模型权重文件路径
    psnr_metrics_all = 0.0
    ssim_metrics_all = 0.0#初始化用于计算所有测试图像的PSNR和SSIM度量的变量。
    model = SRCNN().to(device=device, memory_format=torch.channels_last)#：创建SRCNN模型实例并将其发送到计算设备。

    # 从指定文件加载预训练的SRCNN模型权重。
    checkpoint = torch.load(model_path, map_location=lambda storage, loc: storage)
    model.load_state_dict(checkpoint["state_dict"])

    # 创建用于存储超分辨率实验结果的目录。
    results_dir = os.path.join("srcnn_results")
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)

    # 将模型设置为评估模式，禁用任何在训练期间行为不同的层（如丢弃或批量归一化）
    model.eval()
    # ：开启半精度推理，减少存储中间激活所需的内存量并加速计算。
    model.half()
    # Initialize IQA metrics
    psnr_metrics = 0.0
    ssim_metrics = 0.0

    # 获取所有低分辨率测试图像文件名的排序列表。
    file_names = natsorted(os.listdir(lr_dir))
    # 计算测试图像的总数。
    total_files = len(file_names)

    for index in range(total_files):
        # 获取LR图像路径、SR图像路径、HR图像路径
        lr_image_path = os.path.join(lr_dir, file_names[index])
        sr_image_path = os.path.join(results_dir, f'super_resolution_{file_names[index]}')
        hr_image_path = os.path.join(hr_dir, file_names[index])
        print(f"Processing `{os.path.abspath(lr_image_path)}`...")
        # 读取LR图像和HR图像
        hr_image = cv2.imread(hr_image_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
        #加载低分辨率图像，获取其宽度和高度中较小的值，并使用双三次插值将其缩小4倍。
        lr_img = Image.open(lr_image_path)
        size = np.min(lr_img.size)
        downscale = transforms.Resize(int(size / 4), interpolation=Image.BICUBIC)
        upscale = transforms.Resize(int(size ), interpolation=Image.BICUBIC)
        #将低分辨率图像转换为numpy数组，并将其从RGB格式转换为BGR格式，以便能够保存为图像文件。
        lr_img = downscale(lr_img)
        lr_img=upscale(lr_img)
        lr_image = np.array(lr_img).astype(np.float32) / 255.0
        lr_image = cv2.cvtColor(lr_image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(os.path.join(results_dir,f'GroundTruth_{ file_names[index]}'), hr_image * 255.0)
        cv2.imwrite(os.path.join(results_dir,f'subsample_{ file_names[index]}'), lr_image * 255.0)
        # 获取LR图像和HR图像的Y通道数据
        lr_y_image = bgr2ycbcr(lr_image, True)
        hr_y_image = bgr2ycbcr(hr_image, True)
        # 获取HR图像的Cb和Cr通道数据
        hr_ycbcr_image = bgr2ycbcr(hr_image, False)
        _, hr_cb_image, hr_cr_image = cv2.split(hr_ycbcr_image)
        # 将RGB通道图像数据转换为Tensor通道图像数据
        lr_y_tensor = image2tensor(lr_y_image, False, True).unsqueeze_(0)
        hr_y_tensor = image2tensor(hr_y_image, False, True).unsqueeze_(0)
       # 将Tensor通道图像数据转移到CUDA设备上
        lr_y_tensor = lr_y_tensor.to(device=device, memory_format=torch.channels_last, non_blocking=True)
        hr_y_tensor = hr_y_tensor.to(device=device, memory_format=torch.channels_last, non_blocking=True)
        with torch.no_grad():# 声明一个上下文管理器，以确保在评估模型时不会计算梯度。
            sr_y_tensor = model(lr_y_tensor).clamp_(0, 1.0)#使用预训练的超分辨率模型将低分辨率输入图像(通过lr_y_tensor)转换为高分辨率图像(通过sr_y_tensor)，并将像素值限制在[0,1]范围内。
        sr_y_image = tensor2image(sr_y_tensor, False, True)#将PyTorch张量转换为OpenCV图像格式，并存储生成的高分辨率图像。
        sr_y_image = sr_y_image.astype(np.float32) / 255.0#将像素值标准化为[0,1]的范围内的浮点数。
        sr_ycbcr_image = cv2.merge([sr_y_image, hr_cb_image, hr_cr_image])#将生成的高分辨率亮度通道与原始的彩色通道合并，形成完整的高分辨率图像。
        sr_image = ycbcr2bgr(sr_ycbcr_image)#将YCbCr颜色空间转换回BGR颜色空间，以在屏幕上显示图像。
        hr_y_image = tensor2image(hr_y_tensor, False, True)#将高分辨率目标图像从PyTorch张量转换为OpenCV图像格式。
        hr_y_image = hr_y_image.astype(np.float32) / 255.0#将像素值标准化为[0,1]的范围内的浮点数
        psnr_metrics = psnr(sr_y_tensor, hr_y_tensor).item()#计算峰值信噪比(PSNR)指标，并将其作为Tensor返回。通过.item()方法将其转换为Python float类型。
        ssim_metrics = structural_similarity(sr_y_image, hr_y_image, win_size=11, gaussian_weights=True,
                                             multichannel=True, data_range=1.0, K1=0.01, K2=0.03, sigma=1.5)
        psnr_metrics_all += psnr_metrics
        ssim_metrics_all += ssim_metrics
        text='psnr:'+str(round(float(psnr_metrics), 3))+' ssim:'+str(ssim_metrics)#生成一个文本字符串，包含当前图像的PSNR和SSIM指标值。
        cv2.putText(sr_image, text, (40, 50), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0,255), 1)#在生成的图像上绘制文本字符串。
        print(file_names[index],f' psnr:{psnr_metrics}')
        print(file_names[index], f' ssim:{ssim_metrics}')
        cv2.imwrite(sr_image_path, sr_image * 255.0)
    avg_psnr = 100 if psnr_metrics_all / total_files > 100 else psnr_metrics_all / total_files
    avg_ssim = 1 if ssim_metrics_all / total_files > 1 else ssim_metrics_all / total_files
    print(f"PSNR: {avg_psnr:4.2f} [dB]\n"
          f"SSIM: {avg_ssim:4.4f} [u]")
if __name__ == "__main__":
    main()

Processing `c:\Users\DELL\Desktop\计算机视觉实践\计算机视觉实践-练习3\data\Set5\baby.png`...


C:\Users\DELL\AppData\Local\Temp\ipykernel_70780\1569406781.py:44: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  downscale = transforms.Resize(int(size / 4), interpolation=Image.BICUBIC)
C:\Users\DELL\AppData\Local\Temp\ipykernel_70780\1569406781.py:45: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  upscale = transforms.Resize(int(size ), interpolation=Image.BICUBIC)
C:\Users\DELL\AppData\Local\Temp\ipykernel_70780\1569406781.py:74: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  ssim_metrics = structural_similarity(sr_y_image, hr_y_image, win_size=11, gaussian_weights=True,


baby.png  psnr:33.09375
baby.png  ssim:0.8817651867866516
Processing `c:\Users\DELL\Desktop\计算机视觉实践\计算机视觉实践-练习3\data\Set5\bird.png`...
bird.png  psnr:32.0
bird.png  ssim:0.9054357409477234
Processing `c:\Users\DELL\Desktop\计算机视觉实践\计算机视觉实践-练习3\data\Set5\butterfly.png`...
butterfly.png  psnr:25.140625
butterfly.png  ssim:0.8481625318527222
Processing `c:\Users\DELL\Desktop\计算机视觉实践\计算机视觉实践-练习3\data\Set5\head.png`...
head.png  psnr:32.40625
head.png  ssim:0.7781951427459717
Processing `c:\Users\DELL\Desktop\计算机视觉实践\计算机视觉实践-练习3\data\Set5\woman.png`...
woman.png  psnr:28.4375
woman.png  ssim:0.8778899312019348
PSNR: 30.22 [dB]
SSIM: 0.8583 [u]
